## План
1. Введение 
  * Основные определения
  * Автокорреляция, Стационарность/Нестационарность, Тесть Дьюка-Фуллера
  * Метрики качества прогнозов для временных рядов
2. Сглаживание временного ряда
  * Rolling window estimations
  * Экспоненциальное сглаживание, простое, двойное и Holt-Winters
  * Кросс-валидация на временных рядах
3. Эконометрический подход 
  * Стационарность и единичные корни - еще раз на всякий случай
  * Избавление от нестационарности, дифференцирование, сезонное дифференцирование, выделение трендов
  * ARIMA - выбор начальных параметров модели по ACF, PACF, выбор параметров перебором, построение SARIMAX с подобранными параметрами
  * Ограничения, недостатки эконометрического подхода

# 1) Введение

При работе с временными рядами чаще всего возникает вопрос — а что у нас будет происходить с нашими показателями в ближайший день/неделю/месяц/пр. — сколько будет онлайна, как много действий совершат пользователи, и так далее. К задаче прогнозирования можно подходить по-разному, в зависимости от того, какого качества должен быть проноз, на какой период мы хотим его строить, и, конечно, как долго нужно подбирать и настраивать параметры модели для его получения.

Небольшое [определение](https://ru.wikipedia.org/wiki/Временной_ряд) временного ряда:
> Временной ряд – это последовательность значений $y_{1}$, $y_{2}$, ..., $y_{t}$, где $y_{t} \in R$, то есть значения признаков, которые описывают во времени некий случайный процесс, который измеряется в последовательные постоянные моменты времени (через равные промежутки, постоянные временные интервалы). Очень важно, чтобы временные интервалы между признаками (элементами измерений) были постоянными, иначе задача не сходится к решению временного ряда.

Таким образом, данные оказываются упорядочены относительно неслучайных моментов времени, и, значит, в отличие от случайных выборок, могут содержать в себе дополнительную информацию, которую мы постараемся извлечь. 
----
В целом, задачи, связанные с временными рядами можно разбить на несколько групп
- _**Анализ**_ - где мы сейчас и почему? Куда идем? Скорость движения, волатильность (размах) движения и т.д.
- _**Прогнозирование**_ - когда хотим узнать, что будет дальше и куда мы идем, с какой вероятностью достигнем цели
- _**Поиск аномалий**_ - когда хотим понять, где были проблемы в прошлом или где происходило что-то необычное
- _**Кластеризация и классификация**_ - когда временные ряды сами являются признаками объектов
----

### Пример визуализации временного ряда:
![](./../src/imgs/example_ts.png)

----
### Где применяется анализ и моделирование временных рядов:
- _**Финансовый Сектор (FinTech)**_ - прогнозирование и моделирование движений различных финансовых инструментов, моделирование спроса и предложения и т.д.
- _**Показания датчиков**_ - прогнозирование активности пользователя по данным со смарт-часов, анализ нагрузки на датчики измерения давления и т.д.
- _**Объемы продаж/производства**_
- _**Телеметрия IT-систем**_ - RPS, нагрузки на серверное оборудование, объем запросов и т.д.
- _**Маркетинг и Реклама**_ - анализ поведения рекламных кампаний, окупаемость рекламных кампаний, тренд, сезонность кампаний, ROI и т.д.
- _**Поведенческий Анализ**_ - предсказания социальных активностей, моделирование поведения пользователей в приложении, предсказания оттока, измерение тренда настроений и т.д.
...
----
### Способы визуализации временных рядов:
- Простой линейный график
- Box-Plot
- Candlesticks
- Bar Chart
----

### Составляющие временного ряда:
----
- _**Тренд**_ (T) - плавное долгосрочное изменение уровня временного ряда

![](./../src/imgs/trend_ts.png)
----
- _**Сезонность**_ (S) - циклические изменения уровня временного ряда с постоянным периодом

![](./../src/imgs/trend_seasonality_ts.png)

- _**Цикл**_ (C) - изменения уровня временного ряда с переменным периодом (например, экономические циклы, политические циклы, периоды солнечной активности и т.д.)

![](./../src/imgs/cycles_ts.png)

- _**Случайная (Ненаблюдаемая) Ошибка**_ $\epsilon$ - непрогнозируемая случайная компонента временного ряда


### Практика: чтение и базовый анализ временных рядов.

In [92]:
from mplfinance.original_flavor import candlestick_ohlc # pip install --upgrade mplfinance
import yfinance as yf  # pip install yfinance
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import requests
from datetime import datetime
import pandas as pd
import warnings
import numpy as np
import seaborn as sns


from dateutil.relativedelta import relativedelta # working with dates with style
from scipy.optimize import minimize              # for function minimization
import statsmodels.formula.api as smf            # statistics and econometrics
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from itertools import product
from tqdm import tqdm_notebook

sns.set()
%matplotlib inline
warnings.filterwarnings("ignore")

In [32]:
def get_price_data(ticker='TSLA'):
    tickerData = yf.Ticker(ticker)
    tickerDf = tickerData.history(period='1d', start='2010-1-1', end='2022-9-25')
    return tickerDf.iloc[:, :5]

In [33]:
def plot_candlestick(df, ax=None, fmt="%Y-%m-%d"):
    if ax is None:
        fig, ax = plt.subplots()
    idx_name = df.index.name if df.index.name else 'index'
    dat = df.reset_index()[[idx_name, "Open", "High", "Low", "Close"]]
    dat[idx_name] = dat[idx_name].map(mdates.date2num)
    ax.xaxis_date()
    ax.xaxis.set_major_formatter(mdates.DateFormatter(fmt))
    plt.xticks(rotation=45)
    _ = candlestick_ohlc(ax, dat.values, width=.6, colorup='g', alpha =1)
    ax.set_xlabel(idx_name)
    ax.set_ylabel("OHLC")
    return ax

In [34]:
# get price data (return pandas dataframe)
df = get_price_data()

In [ ]:
df.head()

#### Немного про работу с time series данными:

Когда мы читаем time series данные через Pandas, то по умолчанию колонке с датой присваивается кастомный тип данных, его необходимо перевести соответственно в формат time delta, TimeObj, TimeStamp и т.д.:

In [ ]:
air_quality = pd.read_csv('./../data/air_quality.csv').rename(columns={"date.utc": "datetime"})
air_quality.head()

In [ ]:
air_quality.dtypes

In [ ]:
# Перевод в datetime object для работы с датой:
air_quality['datetime'] = pd.to_datetime(air_quality['datetime'])
air_quality['datetime'].head()

In [ ]:
# Если вы заранее знаете название колонки с датой, то при загрузке вы автоматом можете указать что данная колонка является датой:
air_quality = pd.read_csv('./../data/air_quality.csv', parse_dates=['datetime'])

In [ ]:
# Теперь когда у нас колонка с типом данных TimesTamp мы можем реализовывать различные операции
air_quality['datetime'].min(), air_quality['datetime'].max()

In [ ]:
# Составим новый признак - месяц наблюдения:
air_quality['month'] = air_quality['datetime'].dt.month
air_quality.head()

Группировка и аггрегация данных time series:

In [ ]:
# Например нам необходимо посчитать среднее какого-либо признака для каждого для недели по каждой локации измерений (страна, канал рекламы, город и тд):
air_quality.groupby(
    [air_quality['datetime'].dt.weekday, "location"])['value'].mean()

In [ ]:
# Также можно навесить на это визуализации:
fig, axs = plt.subplots(figsize=(12, 4))
air_quality.groupby(air_quality["datetime"].dt.hour)["value"].mean().plot(
    kind='bar', rot=0, ax=axs)
plt.xlabel("Hour of the day");
plt.ylabel("$NO_2 (µg/m^3)$");

Очень часто для того чтобы эффективно работать с time series данными нам необходимо чтобы колонка с данными даты выступала в качестве индекса, тогда нам будут доступны более продвинутые статистические операции и анализ:

In [ ]:
air_quality_indexed = air_quality.pivot_table(index="datetime", columns="location", values="value", aggfunc="mean")
air_quality_indexed.head()

In [ ]:
air_quality_indexed.index.year, air_quality_indexed.index.weekday

In [ ]:
# Когда у нас дата в качестве индекса задача визуализации по нескольким измерениям (признакам) становится проще:
air_quality_indexed["2019-05-20":"2019-05-21"].plot();

Resampling и Reshaping:

In [ ]:
air_quality_indexed_by_month_max = air_quality_indexed.resample('M').max()
air_quality_indexed_by_month_max

In [ ]:
# Посмотрим по какому периоду проходит семплирование:
air_quality_indexed_by_month_max.index.freq

In [ ]:
air_quality_indexed.resample("D").mean().plot(style="-o", figsize=(10,5))

Полезные материалы по работе (чтнием, аггрегацией, сэмплированием) с time series датафреймами:

------

* [Working With Times Series Data Tutorial](https://towardsdatascience.com/working-with-time-series-data-a8872ebcac3)
* [TS Data Manipulation](https://www.datacamp.com/tutorial/time-series-analysis-tutorial)
* [Выдержка из книги Data Science Handbook по работе с TS](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html)

#### Способы визуализации:

_**Обычный линейный график**_, который вы будете использовать почти всегда:

In [ ]:
data = df['2018-6-15':'2022-6-30']

plt.figure(figsize=(12, 6))
plt.plot(data['Close'], linewidth=5.0)
plt.show()

Кто работал (или будет работать) с финансовыми данными, то очень хорошо подойдет _**Candle Sticks**_:

In [ ]:
fig = plt.figure(figsize=(12,6))
plot_candlestick(data['2022-3-30':'2022-4-30'], fig.subplots())
plt.show()

_**Box-Plot**_ вариант:

In [ ]:
plt.figure(figsize=(12, 6))
plt.boxplot(df['Close'])
plt.show()

Пройдемся немного по базовым вещам используя датасет _monthly-car-sales-in-quebec-1960.csv_:

In [ ]:
df = pd.read_csv('./../data/monthly-car-sales-in-quebec-1960.csv')
df.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['Count'], linewidth=3)
plt.show()

In [51]:
# Выделяем месяц
df['Month'] = df.apply(lambda s: int(s['Month'][-2:]), axis=1)

In [ ]:
# Определим тупо перебором максимальные продажи:
plt.figure(figsize=(12,6))
plt.plot(df['Count'], linewidth=3)
plt.plot(df[df['Month']==5]['Count'], 'ro')
plt.show()

In [ ]:
# Тоже самое только с минимальными продажами:
plt.figure(figsize=(12,6))
plt.plot(df['Count'], linewidth=3)
plt.plot(df[df['Month']==9]['Count'], 'ro')
plt.show()

При помощи графика рассеяния (scatterplot) посмотрите как связаны продажи в текущем месяце и в следующем:

_Текущий месяц к следующему_:

In [ ]:
x = df['Count'][:-1]
y = df['Count'][1:]
plt.figure(figsize=(8,8))
plt.scatter(x,y)
plt.show()

_Текущий месяц к $t+2$_:

In [ ]:
x = df['Count'][:-2]
y = df['Count'][2:]
plt.figure(figsize=(8,8))
plt.scatter(x,y)
plt.show()

_Текущий месяц к $t+5$_:

In [ ]:
x = df['Count'][:-5]
y = df['Count'][5:]
plt.figure(figsize=(8,8))
plt.scatter(x,y)
plt.show()

_Текущий месяц к $t+12$_:

In [ ]:
x = df['Count'][:-12]
y = df['Count'][12:]
plt.figure(figsize=(8,8))
plt.scatter(x,y)
plt.show()

Поговорим немного о такой штуке как автокорреляция:

> Автокорреляция - это статистическая взаимосвязь (коэффициент корреляции) между последовательностями величин (данных, признаков) одного ряда, взятыми с некоторым сдвигом  $ \tau $. Пришло к нам из эконометрики.

Формула:
<br>
$R (\tau) = \frac{E((y_t - E_y)(y_t_+_\tau - E_y))}{D_y}$

<br>
$\rho (\tau) = \frac{\sum_{t=1}^{T-\tau}(y_t - \hat{y})(y_t_+_\tau - \hat{y})}{\sum_{i=1}^{T}(y_t - \hat{y})^2}$


Если у нас сильный коэффициент корреляции, то скорее всего во временном ряду присутствует сезонность.

Как понять какая величина автокорреляции хорошая? А плохая?

Считаем автокорреляцию при помощи метода _pandas - autocorr():_

In [ ]:
for i in range(12):
    print(i, df['Count'].autocorr(i + 1))

Коррелолаграмма - график автокорреляций для разных лагов:

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['Count'])
plt.show()

Как понять какой коэффициент автокорреляции значимый:
1. Просто посмотреть на график автокорреляций и решить (тупо но иногда рабочая схема).
2. Проверить гипотвезу при помощи статистики:
```
Дано: Временной ряд: у(t)
Нулевая гипотеза: H_0 : ro_tau = 0
Альтернативная гипотеза: H_1 : ro_tao != 0
```
Считаем статистику:
<br>
$T(y_t) = \frac{\rho_\tau \sqrt{T - \tau - 2}}{\sqrt{1 - \rho^2_t}}$

Получаем распределение: $T(y_t) \approx S_t (T - \tau - 2)$

Проверяем на P-value: 0.01, 0.05, 0.1 ...

## Познакомимся с временными рядами более конкретно:

In [103]:
ads = pd.read_csv('./../data/ads.csv', index_col=['Time'], parse_dates=['Time'])
currency = pd.read_csv('./../data/currency.csv', index_col=['Time'], parse_dates=['Time'])

Реальные часовые данные по запуску рекламной кампании (х - время, у - количество просмотров в час):

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ads.Ads)
plt.title('Ads watched (hourly data)')
plt.grid(True)
plt.show()

Откопал старенькие данные с позапрошлой работы, это внутриигровые покупки у пользователей в приложении за один день:

In [ ]:
plt.figure(figsize=(15, 7))
plt.plot(currency.GEMS_GEMS_SPENT)
plt.title('In-game currency spendings (daily data)')
plt.grid(True)
plt.show()

## Метрики качества прогноза

Рассмотрим основные и самые распространенные метрики качества прогнозов, которые, по большому счету, являются метриками для задачи регрессии и используются далеко не только во временных рядах.

- [R squared](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination), коэффициент детерминации (в эконометрике - доля объясненной моделью дисперсии), $(-\infty, 1]$

$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$

Очень отсталая и непригодная метрика, которую очень любит менеджмент, который прочитал пару статей про статистику :)

Почему плохая метрика:
1. Неинтерпретируемость - находится в рейндже не от 0 до 1. Поэтому ее иногда трудно объяснить. Почему? Как можно построить модель, которая дает качество хуже чем 0?
2. Метрика не убывает при добавлении новых признаков в признаковое пространство данных. Если мы добавляем любой признак (даже если случайный шум) то метрика не увеличится, а может даже уменьшиться. Поэтому, например, еще и для отбора признаков метрика крайне не пригода.

```python
sklearn.metrics.r2_score
```
---
- [Mean Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error), интерпретируемая метрика, измеряется в тех же единицах, что и исходный ряд, $[0, +\infty)$

$MAE = \frac{\sum\limits_{i=1}^{n} |y_i - \hat{y}_i|}{n}$

Хорошая метрика, понятная, измеряется в тех же единицах, что и изначальный ряд, то есть с масштабом все более или менее понятно. 0 - отлично попали в прогноз и до бесконечности, когда у нас существуют какие-то расхождения.

```python
sklearn.metrics.mean_absolute_error
```
---
- [Median Absolute Error](http://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error), также интерпретируемая метрика, однако её преимущество - нечувствительность (робастность) к выбросам в данных, $[0, +\infty)$

$MedAE = median(|y_1 - \hat{y}_1|, ... , |y_n - \hat{y}_n|)$

Тоже интерпретируемая метрика, которая хорошо работает в сильно зашумленных данных, где много выбросов.

```python
sklearn.metrics.median_absolute_error
```
---
- [Mean Squared Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error), используется в большинстве случаев, сильнее наказывает модель за большие ошибки и меньше - за маленькие (парабола), $[0, +\infty)$

$MSE = \frac{1}{n}\sum\limits_{i=1}^{n} (y_i - \hat{y}_i)^2$

Ее и так вы знаете.

```python
sklearn.metrics.mean_squared_error
```
---
- [Mean Squared Logarithmic Error](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-logarithmic-error), практически тоже самое, но значения предварительно логарифмируются, таким образом маленьким ошибкам также уделяется значительное внимание, обычно используется, если данным присущ экспоненциальный рост, $[0, +\infty)$

$MSLE = \frac{1}{n}\sum\limits_{i=1}^{n} (log(1+y_i) - log(1+\hat{y}_i))^2$

Почти тоже самое что и MSE, только мы добавляем еще логарифм. Важно если используете, убедитесь что ряд не околонулевой. Зачем она нужна? Логарифмирование временного ряда очень важная операция (как увидим впоследующем) в том случае, когда у нас есть какой-либо сильный экспоненциальный рост. Позволяет избавиться от некоторых допущений.

```python
sklearn.metrics.mean_squared_log_error
```
---
- Mean Absolute Percentage Error, как MAE, только в процентах, - удобно для объяснения заказчику качества прогноза, $[0, +\infty)$

$MAPE = \frac{100}{n}\sum\limits_{i=1}^{n} \frac{|y_i - \hat{y}_i|}{y_i}$

Очень хорошая метрика, супер интерпретируемая. Измеряется в %. Если MAPE = 10 то вы в среднем ошибаетесь на 10%. Стоит помнить про нее, но ее не удобно оптимизировать, особенно если вы используете ее в нейронной сети.

```python
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
```

In [106]:
# импортируем всё, о чем говорили выше

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Отлично, теперь мы знаем, как можно измерить качество нашего прогноза, какие метрики стоит применять и как объяснить всё заказчику, дело осталось за малым - нужно построить прогноз

# 2) Сглаживаем, оцениваем и прогнозируем:

Начнем моделирование с наивного предположения - "завтра будет, как вчера", но вместо модели вида $\hat{y}_{t} = y_{t-1}$ будем считать, что будущее значение переменной зависит от среднего $n$ её предыдущих значений, а значит, воспользуемся скользящей средней. 

$$\hat{y}_{t} = \frac{1}{k} \displaystyle\sum^{k-1}_{n=0} y_{t-n}$$

In [ ]:
def moving_average(series, n):
    """
        Calculate average of last n observations
    """
    return np.average(series[-n:])

moving_average(ads, 24) # prediction for the last observed day (past 24 hours)

К сожалению, такой прогноз долгосрочным сделать не удастся, для получения предсказания на шаг вперед предыдущее значение должно быть фактически наблюдаемой величиной.

Зато у скользящей средней есть другое применение - сглаживание исходного ряда для выявления трендов, в pandas есть готовая реализация - [`DataFrame.rolling(window).mean()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html). Чем больше зададим ширину интервала - тем более сглаженным окажется тренд. В случае, если данные сильно зашумлены, что особенно часто встречается, например, в финансовых показателях, такая процедура может помочь увидеть общие паттерны.

In [108]:
def plotMovingAverage(series, window, plot_intervals=False, scale=1.95, plot_anomalies=False):

    """
        series - dataframe with timeseries
        window - rolling window size 
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 

    """
    # Считаем скользящую среднюю по нашему временному ряду используя тот метод Pandas про который было сказано выше
    rolling_mean = series.rolling(window=window).mean()
    plt.figure(figsize=(15,5))
    plt.title("Moving average\n window size = {}".format(window))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")

    # Теперь необходимо посчитать доверительные интервалы:
    if plot_intervals:
        # Считаем метрику ошибки например среднюю абсолютную ашибку MAE так как те же единицы измерения что и исходный ряд
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        # Возьмем посчитаем скользящее отклонение, то есть фактически доверительный интервал
        deviation = np.std(series[window:] - rolling_mean[window:])
        # Для нижней границы возьмем наш сглаженный ряд вычтем из него ошибку + стандартное отклонение
        lower_bond = rolling_mean - (mae + scale * deviation)
        # Аналогично для верхней границы только сложим все что посчитали
        upper_bond = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bond, "r--", label="Upper Bond / Lower Bond")
        plt.plot(lower_bond, "r--")
        
        # Having the intervals, find abnormal values
        if plot_anomalies:
            # Просто сравнение на значения верхней и нижней границы чтобы определить является ли наблюдение аномальным:
            anomalies = pd.DataFrame(index=series.index, columns=series.columns)
            anomalies[series<lower_bond] = series[series<lower_bond]
            anomalies[series>upper_bond] = series[series>upper_bond]
            plt.plot(anomalies, "ro", markersize=10)
        
    plt.plot(series[window:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)

Применим такое сглаживание к нашим рядам:

Сглаживание по последним 4 часам

In [ ]:
plotMovingAverage(ads, 4)

По 12 часам (полдня)

In [ ]:
plotMovingAverage(ads, 12) 

Сглаживание по 24 часам (одни сутки) - выделяется дневной тренд

In [ ]:
plotMovingAverage(ads, 24)

Также можно отобразить доверительные интервалы для наших средних значений

In [ ]:
plotMovingAverage(ads, 4, plot_intervals=True)

А теперь попробуем при помощи этих доверительных интервалов поймать аномалию во временном ряде. К сожалению или к счастью, конкретно в этом ряду всё более-менее хорошо и гладко, поэтому аномальное значение мы зададим самостоятельно в датафрейме `ads_anomaly`

In [112]:
ads_anomaly = ads.copy()
ads_anomaly.iloc[-20] = ads_anomaly.iloc[-20] * 0.2 # допустим что у нас появится дроп в рекламе который упал на 80%

И посмотрим, удастся ли таким простым образом её поймать

In [ ]:
plotMovingAverage(ads_anomaly, 4, plot_intervals=True, plot_anomalies=True)

Тоже самое можно проделать и со вторым рядом, попробуем сгладить его с окном в 7 дней, то есть по неделям, и поймать аномалии

In [ ]:
plotMovingAverage(currency, 7, plot_intervals=True, plot_anomalies=True) # недельное сглаживание

_Вопрос на понимание_: __Когда такая система даст сбой? :)__

Модификацией простой скользящей средней является взвешенная средняя, внутри которой наблюдениям придаются различные веса, в суме дающие единицу, при этом обычно последним наблюдениям присваивается больший вес. 


$$\hat{y}_{t} = \displaystyle\sum^{k}_{n=1} \omega_n y_{t+1-n}$$

In [15]:
def weighted_average(series, weights):
    """
        Calculate weighter average on series
    """
    result = 0.0
    weights.reverse()
    for n in range(len(weights)):
        result += series.iloc[-n-1] * weights[n]
    return float(result)

In [ ]:
weighted_average(ads, [0.6, 0.3, 0.1])

## Экспоненциальное сглаживание

А теперь посмотрим, что произойдёт, если вместо взвешивания последних $n$ значений ряда мы начнем взвешивать все доступные наблюдения, при этом экспоненциально уменьшая веса по мере углубления в исторические данные. В этом нам поможет формула простого [экспоненциального сглаживания](http://www.machinelearning.ru/wiki/index.php?title=Экспоненциальное_сглаживание):

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1} $$

Здесь модельное значение представляет собой средневзвешенную между текущим истинным и предыдущим модельным значениями. Вес $\alpha$ называется сглаживающим фактором. Он определяет, как быстро мы будем "забывать" последнее доступное истинное наблюдение. Чем меньше $\alpha$, тем больше влияния оказывают предыдущие модельные значения, и тем сильнее сглаживается ряд. 

Экспоненциальность скрывается в рекурсивности функции - каждый раз мы умножаем $(1-\alpha)$ на предыдущее модельное значение, которое, в свою очередь, также содержало в себе $(1-\alpha)$, и так до самого начала.

In [115]:
def exponential_smoothing(series, alpha):
    """
        series - dataset with timestamps
        alpha - float [0.0, 1.0], smoothing parameter
    """
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result

In [116]:
def plotExponentialSmoothing(series, alphas):
    """
        Plots exponential smoothing with different alphas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters
        
    """
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(15, 7))
        for alpha in alphas:
            plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
        plt.plot(series.values, "c", label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Exponential Smoothing")
        plt.grid(True);

In [ ]:
plotExponentialSmoothing(ads.Ads, [0.3, 0.05])

In [ ]:
plotExponentialSmoothing(currency.GEMS_GEMS_SPENT, [0.3, 0.05])

## Двойное экспоненциальное сглаживание

До сих пор мы могли получить от наших методов в лучшем случае прогноз лишь на одну точку вперёд (и ещё красиво сгладить ряд), это здорово, но недостаточно, поэтому переходим к расширению экспоненциального сглаживания, которое позволит строить прогноз сразу на две точки вперед (и тоже красиво сглаживать ряд).

В этом нам поможет разбиение ряда на две составляющие - уровень (level, intercept) $\ell$ и тренд $b$ (trend, slope). Уровень, или ожидаемое значение ряда, мы предсказывали при помощи предыдущих методов, а теперь такое же экспоненциальное сглаживание применим к тренду, наивно или не очень полагая, что будущее направление изменения ряда зависит от взвешенных предыдущих изменений.

$$\ell_x = \alpha y_x + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$\hat{y}_{x+1} = \ell_x + b_x$$

В результате получаем набор функций. Первая описывает уровень - он, как и прежде, зависит от текущего значения ряда, а второе слагаемое теперь разбивается на предыдущее значение уровня и тренда. Вторая отвечает за тренд - он зависит от изменения уровня на текущем шаге, и от предыдущего значения тренда. Здесь в роли веса в экспоненциальном сглаживании выступает коэффициент $\beta$. Наконец, итоговое предсказание представляет собой сумму модельных значений уровня и тренда.

In [119]:
def double_exponential_smoothing(series, alpha, beta):
    """
        series - dataset with timeseries
        alpha - float [0.0, 1.0], smoothing parameter for level
        beta - float [0.0, 1.0], smoothing parameter for trend
    """
    # first value is same as series
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

def plotDoubleExponentialSmoothing(series, alphas, betas):
    """
        Plots double exponential smoothing with different alphas and betas
        
        series - dataset with timestamps
        alphas - list of floats, smoothing parameters for level
        betas - list of floats, smoothing parameters for trend
    """
    
    with plt.style.context('seaborn-white'):    
        plt.figure(figsize=(20, 8))
        for alpha in alphas:
            for beta in betas:
                plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
        plt.plot(series.values, label = "Actual")
        plt.legend(loc="best")
        plt.axis('tight')
        plt.title("Double Exponential Smoothing")
        plt.grid(True)

In [ ]:
plotDoubleExponentialSmoothing(ads.Ads, alphas=[0.9, 0.02], betas=[0.9, 0.02])

In [ ]:
plotDoubleExponentialSmoothing(currency.GEMS_GEMS_SPENT, alphas=[0.9, 0.02], betas=[0.9, 0.02])

Теперь у нас появилась возможность настраивать уже два параметра - $\alpha$ и $\beta$. Первый отвечает за сглаживание ряда вокруг тренда, второй - за сглаживание самого тренда. Чем выше значения, тем больший вес будет отдаваться последним наблюдениям и тем менее сглаженным окажется модельный ряд. Комбинации параметров могут выдавать достаточно причудливые результаты, особенно если задавать их руками. 

## Тройное экспоненциальное сглаживание a.k.a. Holt-Winters (1970):

Итак, успешно добрались до следующего варианта экспоненциального сглаживания, на сей раз тройного.

Идея этого метода заключается в добавлении еще одной, третьей, компоненты - сезонности. Сезонная компонента в модели будет объяснять повторяющиеся колебания вокруг уровня и тренда, а характеризоваться она будет длиной сезона - периодом, после которого начинаются повторения колебаний. Для каждого наблюдения в сезоне формируется своя компонента, например, если длина сезона составляет 7 (например, недельная сезонность), то получим 7 сезонных компонент, по штуке на каждый из дней недели.

Получаем новую систему:

$$\ell_x = \alpha(y_x - s_{x-L}) + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$s_x = \gamma(y_x - \ell_x) + (1-\gamma)s_{x-L}$$

$$\hat{y}_{x+m} = \ell_x + mb_x + s_{x-L+1+(m-1)modL}$$

Уровень теперь зависит от текущего значения ряда за вычетом соответствующей сезонной компоненты, тренд остаётся без изменений, а сезонная компонента зависит от текущего значения ряда за вычетом уровня и от предыдущего значения компоненты. При этом компоненты сглаживаются через все доступные сезоны, например, если это компонента, отвечающая за понедельник, от и усредняться она будет только с другими понедельниками. Подробнее про работу усреднений и оценку начальных значений тренда и сезонных компонент можно почитать [здесь](http://www.itl.nist.gov/div898/handbook/pmc/section4/pmc435.htm). Теперь, имея сезонную компоненту, мы можем предсказывать уже не на один, и даже не на два, а на произвольные $m$ шагов вперёд, что не может не радовать. 

Ниже импортируем класс для построения модели тройного экспоненциального сглаживания, также известного по фамилиям её создателей - Чарльза Хольта и его студента Питера Винтерса. 
Дополнительно в модель включен метод Брутлага для построения доверительных интервалов:

$$\hat y_{max_x}=\ell_{x−1}+b_{x−1}+s_{x−T}+m⋅d_{t−T}$$

$$\hat y_{min_x}=\ell_{x−1}+b_{x−1}+s_{x−T}-m⋅d_{t−T}$$

$$d_t=\gamma∣y_t−\hat y_t∣+(1−\gamma)d_{t−T}$$

где $T$ - длина сезона, $d$ - предсказанное отклонение, а остальные параметры берутся из тройного сглаживани. Подробнее о методе и о его применении к поиску аномалий во временных рядах можно прочесть [здесь](https://fedcsis.org/proceedings/2012/pliks/118.pdf)

In [122]:
# вроде есть в statsmodels или sklearn...
class HoltWinters:
    """
    Holt-Winters model with the anomalies detection using Brutlag method
    # series - initial time series
    # slen - length of a season
    # alpha, beta, gamma - Holt-Winters model coefficients
    # n_preds - predictions horizon
    # scaling_factor - sets the width of the confidence interval by Brutlag (usually takes values from 2 to 3)
    """

    def __init__(self, series, slen, alpha, beta, gamma, n_preds, scaling_factor=1.96):
        self.series = series
        self.slen = slen
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.n_preds = n_preds
        self.scaling_factor = scaling_factor

    def initial_trend(self):
        sum = 0.0
        for i in range(self.slen):
            sum += float(self.series[i+self.slen] - self.series[i]) / self.slen
        return sum / self.slen  
    
    def initial_seasonal_components(self):
        seasonals = {}
        season_averages = []
        n_seasons = int(len(self.series)/self.slen)
        # let's calculate season averages
        for j in range(n_seasons):
            season_averages.append(sum(self.series[self.slen*j:self.slen*j+self.slen])/float(self.slen))
        # let's calculate initial values
        for i in range(self.slen):
            sum_of_vals_over_avg = 0.0
            for j in range(n_seasons):
                sum_of_vals_over_avg += self.series[self.slen*j+i]-season_averages[j]
            seasonals[i] = sum_of_vals_over_avg/n_seasons
        return seasonals
          
    def triple_exponential_smoothing(self):
        self.result = []
        self.Smooth = []
        self.Season = []
        self.Trend = []
        self.PredictedDeviation = []
        self.UpperBond = []
        self.LowerBond = []
        
        seasonals = self.initial_seasonal_components()
        
        for i in range(len(self.series)+self.n_preds):
            if i == 0: # components initialization
                smooth = self.series[0]
                trend = self.initial_trend()
                self.result.append(self.series[0])
                self.Smooth.append(smooth)
                self.Trend.append(trend)
                self.Season.append(seasonals[i%self.slen])
                
                self.PredictedDeviation.append(0)
                
                self.UpperBond.append(self.result[0] + 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                
                self.LowerBond.append(self.result[0] - 
                                      self.scaling_factor * 
                                      self.PredictedDeviation[0])
                continue
                
            if i >= len(self.series): # predicting
                m = i - len(self.series) + 1
                self.result.append((smooth + m*trend) + seasonals[i%self.slen])
                
                # when predicting we increase uncertainty on each step
                self.PredictedDeviation.append(self.PredictedDeviation[-1]*1.01) 
                
            else:
                val = self.series[i]
                last_smooth, smooth = smooth, self.alpha*(val-seasonals[i%self.slen]) + (1-self.alpha)*(smooth+trend)
                trend = self.beta * (smooth-last_smooth) + (1-self.beta)*trend
                seasonals[i%self.slen] = self.gamma*(val-smooth) + (1-self.gamma)*seasonals[i%self.slen]
                self.result.append(smooth+trend+seasonals[i%self.slen])
                
                # Deviation is calculated according to Brutlag algorithm.
                self.PredictedDeviation.append(self.gamma * np.abs(self.series[i] - self.result[i]) 
                                               + (1-self.gamma)*self.PredictedDeviation[-1])
                     
            self.UpperBond.append(self.result[-1] + 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.LowerBond.append(self.result[-1] - 
                                  self.scaling_factor * 
                                  self.PredictedDeviation[-1])

            self.Smooth.append(smooth)
            self.Trend.append(trend)
            self.Season.append(seasonals[i%self.slen])

## Кросс-валидация на временных рядах

Перед тем, как построить модель, поговорим, наконец, о не ручной оценке параметров для моделей. 

Ничего необычного здесь нет, по-прежнему сначала необходимо выбрать подходящуюю для данной задачи функцию потерь, которая будет следить за качеством подгонки модели под исходные данные. Затем будем оценивать на кросс-валидации значение функции потерь при данных параметрах модели, искать градиент, менять в соответствии с ним параметры и бодро опускаться в сторону глобального минимума ошибки. 

Так как временной ряд внезапно имеет временную структуру, случайно перемешивать в фолдах значения всего ряда без сохранения этой структуры нельзя, иначе в процессе потеряются все взаимосвязи наблюдений друг с другом. Поэтому придется использовать чуть более хитрый способ для оптимизации параметров, официального названия которому я так и не нашел, но на сайте [CrossValidated](https://stats.stackexchange.com/questions/14099/using-k-fold-cross-validation-for-time-series-model-selection), предлагают название "cross-validation on a rolling basis", что не дословно можно перевести как кросс-валидация на скользящем окне.

Суть достаточно проста - начинаем обучать модель на небольшом отрезке временного ряда, от начала до некоторого $t$, делаем прогноз на $t+n$ шагов вперед и считаем ошибку. Далее расширяем обучающую выборку до $t+n$ значения и прогнозируем с $t+n$ до $t+2*n$, так продолжаем двигать тестовый отрезок ряда до тех пор, пока не упрёмся в последнее доступное наблюдение. В итоге получим столько фолдов, сколько $n$ уместится в промежуток между изначальным обучающим отрезком и всей длиной ряда.

<img src="https://habrastorage.org/files/f5c/7cd/b39/f5c7cdb39ccd4ba68378ca232d20d864.png"/>

Теперь, зная, как настраивать кросс-валидацию, подберём с её помощью параметры для модели Хольта-Винтерса

In [123]:
from sklearn.model_selection import TimeSeriesSplit

def timeseriesCVscore(params, series, loss_function=mean_squared_error, slen=24):
    """
        Returns error on CV
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # errors array
    errors = []
    values = series.values
    alpha, beta, gamma = params
    # set the number of folds for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)
    # iterating over folds, train model on each, forecast and calculate error
    for train, test in tscv.split(values):
        model = HoltWinters(series=values[train], slen=slen, 
                            alpha=alpha, beta=beta, gamma=gamma, n_preds=len(test))
        model.triple_exponential_smoothing()
        predictions = model.result[-len(test):]
        actual = values[test]
        error = loss_function(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

В модели Хольта-Винтерса, так же, как и в других моделях экспоненциального сглаживания, есть ограничение на величину сглаживающих коэффициентов. Каждый из них должен быть от 0 до 1. Соответственно, чтобы минимизировать нашу функцию потерь, нам нужно использовать метод, который поддерживает ограничения на оцениваемые параметры. В нашем случае это метод truncated Newton conjugate gradient

In [ ]:
%%time
data = ads.Ads[:-20] # leave some data for testing
# initializing model parameters alpha, beta and gamma
x = [0, 0, 0]
# Minimizing the loss function 
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_squared_error), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)
# ...and train the model with them, forecasting for the next 50 hours
model = HoltWinters(data, slen = 24, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 50, scaling_factor = 3)
model.triple_exponential_smoothing()

In [125]:
def plotHoltWinters(series, plot_intervals=False, plot_anomalies=False):
    """
        series - dataset with timeseries
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies 
    """
    plt.figure(figsize=(20, 10))
    plt.plot(model.result, label = "Model")
    plt.plot(series.values, label = "Actual")
    error = mean_absolute_percentage_error(series.values, model.result[:len(series)])
    plt.title("Mean Absolute Percentage Error: {0:.2f}%".format(error))
    
    if plot_anomalies:
        anomalies = np.array([np.NaN]*len(series))
        anomalies[series.values<model.LowerBond[:len(series)]] = \
            series.values[series.values<model.LowerBond[:len(series)]]
        anomalies[series.values>model.UpperBond[:len(series)]] = \
            series.values[series.values>model.UpperBond[:len(series)]]
        plt.plot(anomalies, "o", markersize=10, label = "Anomalies")
    
    if plot_intervals:
        plt.plot(model.UpperBond, "r--", alpha=0.5, label = "Up/Low confidence")
        plt.plot(model.LowerBond, "r--", alpha=0.5)
        plt.fill_between(x=range(0,len(model.result)), y1=model.UpperBond, 
                         y2=model.LowerBond, alpha=0.2, color = "grey")    
        
    plt.vlines(len(series), ymin=min(model.LowerBond), ymax=max(model.UpperBond), linestyles='dashed')
    plt.axvspan(len(series)-20, len(model.result), alpha=0.3, color='lightgrey')
    plt.grid(True)
    plt.axis('tight')
    plt.legend(loc="best", fontsize=13);

In [ ]:
plotHoltWinters(ads.Ads)

In [ ]:
plotHoltWinters(ads.Ads, plot_intervals=True, plot_anomalies=True)

Если посмотреть на смоделированное отклонение, хорошо видно, что модель достаточно резко регирует на значительные изменения в структуре ряда, но при этом быстро возвращает дисперсию к обычным значениям, "забывая" прошлое. Такая особенность позволяет неплохо и без значительных затрат на подготовку-обучение модели настроить систему по детектированию аномалий даже в достаточно шумных рядах.

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis('tight')
plt.title("Brutlag's predicted deviation");

Прогоним тот же самый алгоритм для второго временного ряда, в котором, как мы знаем, есть тренд и месячная сезонность

In [ ]:
%%time
data = currency.GEMS_GEMS_SPENT[:-50] 
slen = 30 # 30-day seasonality

x = [0, 0, 0]
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_absolute_percentage_error, slen), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1))
              )

alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

model = HoltWinters(data, slen = slen, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 100, scaling_factor = 3)
model.triple_exponential_smoothing()

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT)

Выглядит весьма правдоподобно, модель уловила и восходящий тренд, и сезонные пики, и в целом достаточно точно описывает наш временной ряд

In [ ]:
plotHoltWinters(currency.GEMS_GEMS_SPENT, plot_intervals=True, plot_anomalies=True)

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis('tight')
plt.title("Brutlag's predicted deviation");

# 3) Эконометрический подход

### Стационарность

Перед тем, как перейти к моделированию, стоит сказать о таком важном свойстве временного ряда, как [**стационарность**](https://ru.wikipedia.org/wiki/Стационарность). 
Под стационарностью понимают свойство процесса не менять своих статистических характеристик с течением времени, а именно постоянство матожидания, постоянство дисперсии (она же [гомоскедастичность](https://ru.wikipedia.org/wiki/Гомоскедастичность)) и независимость ковариационной функции от времени (должна зависеть только от расстояния между наблюдениями). Наглядно можно посмотреть на эти свойства на картинках, взятых из поста [Sean Abu](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/):

- Временной ряд справа не является стационарным, так как его матожидание со временем растёт

<img src="https://habrastorage.org/files/20c/9d8/a63/20c9d8a633ec436f91dccd4aedcc6940.png"/>

- Здесь не повезло с дисперсией - разброс значений ряда существенно варьируется в зависимости от периода

<img src="https://habrastorage.org/files/b88/eec/a67/b88eeca676d642449cab135273fd5a95.png"/>

- Наконец, на последнем графике видно, что значения ряда внезапно становятся ближе друг ко другу, образуя некоторый кластер, а в результате получаем непостоянство ковариаций

<img src="https://habrastorage.org/files/2f6/1ee/cb2/2f61eecb20714352840748b826e38680.png"/>

Почему стационарность так важна? По стационарному ряду просто строить прогноз, так как мы полагаем, что его будущие статистические характеристики не будут отличаться от наблюдаемых текущих. Большинство моделей временных рядов так или иначе моделируют и предсказывают эти характеристики (например, матожидание или дисперсию), поэтому в случае нестационарности исходного ряда предсказания окажутся неверными. К сожалению, большинство временных рядов, с которыми приходится сталкиваться за пределыми учебных материалов, стационарными не являются, но с этим можно (и нужно) бороться.

<br>

Есть несколько вариантов стационарности:
1. Строгая стационарность - ряд y(t) нзывается строго стационарным, если для любого s совместное распределение верояностей y(t), y(t+1),..., y(y+k) совпадает с совместным распределением вероятностей y(t+s), y(t+s+1),...,y(y+s+k).
2. Слабая стационарность ряда y(t) если E(y(t)) = cont и cov(y(t), y(t+s)) зависит только от s

Важное правило: Всякий строго стационарный ряд так же и слабо стационарен.

Зачем все эти разговоры про стационарность:

* Стационарный ряд легче предсказывать. Можно предположить, что статистические свойства распределения не изменяться.
* В большинстве моделей временных рядов закладывается гипотеза о стационарности временного ряда. Это означает, что получаемые оценки будут надежными только в случае стационарности временного ряда.


Стационарность:
* Тренд --> нестационарность
* Сезонность --> нестационарность
* Цикл --> ?



### Интуитивное понимание:

![](./../src/imgs/stationarity.png)

Как превратить ряд в стационарный:
1. Для рядов с монотонно меняющейся дисперсией - логарифмирование
2. Дифференцирование – переход к попарным разностям. Позволяет избавиться от тренда
3. Сезонное дифференцирование – вычитаем не предыдущее значение, а сдвинутое на сезон
4. Дифференцировать можно несколько раз подряд

![](./../src/imgs/stat_to_no_stat.png)

Чтобы бороться с нестационарностью, нужно узнать её в лицо, потому посмотрим, как её детектировать. Для этого обратимся к белому шуму и случайному блужданию, чтобы выяснить как попасть из одного в другое.


График белого шума:

In [ ]:
white_noise = np.random.normal(size=1000)
with plt.style.context('bmh'):  
    plt.figure(figsize=(15, 5))
    plt.plot(white_noise)

Итак, процесс, порожденный стандартным нормальным распределением, стационарен, колеблется вокруг нуля с отклонением в 1. Теперь на основании него сгенерируем новый процесс, в котором каждое последующее значение будет зависеть от предыдущего: $x_t = \rho x_{t-1} + e_t$

In [ ]:
def plotProcess(n_samples=1000, rho=0):
    x = w = np.random.normal(size=n_samples)
    for t in range(n_samples):
        x[t] = rho * x[t-1] + w[t]

    with plt.style.context('bmh'):  
        plt.figure(figsize=(10, 3))
        plt.plot(x)
        plt.title("Rho {}\n Dickey-Fuller p-value: {}".format(rho, round(sm.tsa.stattools.adfuller(x)[1], 3)))
        
for rho in [0, 0.6, 0.9, 1]:
    plotProcess(rho=rho)

На первом графике получился точно такой же стационарный белый шум, который строился раньше. На втором значение  увеличилось до $0.6$, в результате чего на графике стали появляться более широкие циклы, но в целом стационарным он быть пока не перестал. Третий график всё сильнее отклоняется от нулевого среднего значения, но всё ещё колеблется вокруг него. Наконец, значение  равное единице дало процесс случайного блуждания — ряд не стационарен.


Происходит это из-за того, что при достижении критической единицы, ряд $x_t = \rho x_{t-1} + e_t$ перестаёт возвращаться к своему среднему значению. Если вычесть из левой и правой части $x_{t-1}$, то получим  $x_t - x_{t-1} = (\rho - 1) x_{t-1} + e_t$, где выражение слева — первые разности. Если $\rho=1$, то первые разности дадут стационарный белый шум $e_t$. Этот факт лёг в основу теста Дики-Фуллера на стационарность ряда (наличие единичного корня). Если из нестационарного ряда первыми разностями удаётся получить стационарный, то он называется интегрированным первого порядка. Нулевая гипотеза теста — ряд не стационарен, отвергалась на первых трех графиках, и принялась на последнем. Стоит сказать, что не всегда для получения стационарного ряда хватает первых разностей, так как процесс может быть интегрированным с более высоким порядком (иметь несколько единичных корней), для проверки таких случаев используют расширенный тест Дики-Фуллера, проверяющий сразу несколько лагов.


Бороться с нестационарностью можно множеством способов — разностями различного порядка, выделением тренда и сезонности, сглаживаниями и преобразованиями, например, Бокса-Кокса или логарифмированием.

### Практика: Избавляемся от нестационарности:

In [ ]:
df = pd.read_csv('./../data/monthly-sales-of-company-x-jan-6.csv')
plt.figure(figsize=(12,6))
plt.plot(df['Count'])
plt.show()

In [ ]:
autocorrelation_plot(df['Count'])
plt.show()

Нормализуйте дисперсию временного ряда (scipy boxcox):

In [ ]:
from scipy.stats import boxcox
modified = boxcox(df['Count'], 0)
plt.figure(figsize=(12,6))
plt.plot(modified)
plt.show()

Продифференцируйте на n сдвиг:

In [ ]:
import numpy as np
modified = np.diff(modified, 1)
plt.figure(figsize=(12,6))
plt.plot(modified)
plt.show()

In [ ]:
autocorrelation_plot(modified)
plt.show()

Тест Дики-Фуллера:

In [ ]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [ ]:
test_stationarity(df['Count'])

In [ ]:
test_stationarity(modified)

## Полноценный Анализ Временного ряда:


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from scipy.stats import boxcox
%matplotlib inline

In [ ]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
    for [key, value] in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)

In [ ]:
def tsplot(y, lags=None, figsize=(14, 8), style='bmh'):
    test_stationarity(y)
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):
        plt.figure(figsize=figsize)
        layout = (4, 1)
        ts_ax = plt.subplot2grid(layout, (0, 0), rowspan=2)
        acf_ax = plt.subplot2grid(layout, (2, 0))
        pacf_ax = plt.subplot2grid(layout, (3, 0))

        y.plot(ax=ts_ax, color='blue', label='Or')
        ts_ax.set_title('Original')

        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)

        plt.tight_layout()
    return

In [ ]:
series = pd.read_csv("./../data/international-airline-passengers.csv")['Count']

In [ ]:
tsplot(series)

Как мы видим, и тест Дики-Фуллера и графики коррелограмм не отвергают гипотезу о нестационарности ряда. Для начала уберем изменение дисперсии при помощи преобразования Бокса-Кокса

In [ ]:
series = boxcox(series, 0)

In [ ]:
tsplot(series)

Нам удалось убрать размах дисперсии, но тест Дикки-Фуллера все еще не отвергает гипотезу о нестационарности ряда. По графику ряда видно наличие сильного тренда. Уберем его дифференцированием.

In [ ]:
series = series[1:] - series[:-1]

In [ ]:
tsplot(series)

Стало еще лучше, но по графику коррелограммы видно сильное влияние сезонности. Уберем ее

In [ ]:
series = series[12:] - series[:-12]

In [ ]:
tsplot(series)

Теперь тест Дики-Фуллера и графики коррелограмм отвергают гипотезу о нестационарности ряда!